# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования**

В результате исследования необходимо ответить на следующие вопросы:

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные представлены в файле data.csv. О качестве данных ничего не известно, поэтому исследование будет состоять из трёх этапов:
1. Обзор данных.
2. Предобрадотка данных.
3. Ответы на поставленные вопросы.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Составим первое представление о данных платёжеспособности клиентов.

Импортруем библиотеку pandas

In [142]:
#импортируем библиотеку pandas
import pandas as pd

Прочитаем файл data.csv из папки /datasets и запишем его в переменную df

In [143]:
#чтение из файла data.csv и запись в переменную df
df = pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк датафрейма data.csv

In [144]:
#получение первых 10 строк таблицы
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о датафрейме df.

In [145]:
# Вывод общей информации по датафрейму
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Также изучим датафрейм с помощью функции .describe().

In [146]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [147]:
#выведем уникальные занчения из столбца gender
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

**Первичный анализ данных**

Датафрейм df состоит из двенадцати столбцов:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

В названиях столбцов нет нарушения стиля, поэтому нет необходимости их переименовывать.

В датафрейме присутствуют некоторые проблемы, которые осложняют анализ данных:

- В столбцах `days_employed` и `total_income` существуют пропуски значений.

- В столбце `education` присутствуют неявные дубликаты.

- В столбце `days_employed`, `dob_years` и `gender` содержатся подозрительные данные.

- Тип float в столбцах `days_employed` и `total_income` усложняет визуальный анализ данных.


**Вывод**

В каждой строке таблицы — данные платежеспособности клиентов.

Предварительно, можно утверждать, что данных достаточно для ответов на поставленные вопросы. Однако, для корректного анализа необходимо выполнить следующую предоработку данных и их категорирование. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Во-первых, определим количество пропусков в каждом столбце. Для этого применим метод .isna() в паре с методом sum().

In [148]:
#подсчет количества пропусков и вывод в виде фрейма
df.isna().sum().to_frame()

,0
children,0
days_employed,2174
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


Странно, что в столбцах `days_employed` и `total_income` одинаковое количество пропусков (2174). Возможно, пропуски в этих столбцах являются связанными. Необходимо проверить эту гипотезу.

In [149]:
#вывод первых 10 строк, в которых значение days_employed равно NaN
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Видно, что в первых 10 строках, в которых значение `days_employed` равно NaN, значение `total_income` также равно NaN, но выводы необходимо делать по всей выборке, а не по первым 10 строкам. Поэтому подсчитаем количество строк, в которых значения в столбцах `days_employed` и `total_income` равны NaN.

In [150]:
#вывод количества строк, в которых days_employed и total_income равны NaN
len(df[df['days_employed'].isna() & df['total_income'].isna()])

2174

Из вычисления выше следует, что пропуски в столбцах `days_employed` и `total_income` являются связанными. При этом связь с другими столбцами не наблюдается. Возможно, люди просто не захотели указывать свой рабочий стаж и размер заработной платы.

Количество строк, в которых существуют пропуски, составляет порядка 10% от общего количества строк датафрейма. Это большое количество, поэтому удаление этих строк из общей выборки может сказаться на итоговых результатах исследования, например, для вопросов о зависимости между наличием детей/семейным положением и возвратом кредита в срок. Следовательно, эти строки не стоит удалять из общей выборки.

Значения в столбцах `days_employed` и `total_income` являются количественными, поэтому их можно заменить на среднее арифметическое или на медиану. Попробуем произвести оптимальный выбор между средним арифметическим и медианой для каждого проблемного столбца.

In [151]:
#вычисление среднего арифметического для столбца days_employed
df['days_employed'].mean()

63046.49766147338

In [152]:
#вычисление медианы для столбца days_employed
df['days_employed'].median()

-1203.369528770489

Значения среднего арифметического и медианы столбца `days_employed` сильно разнятся. Это значит в данном столбце существуют значения, которые сильно больше многих других. Тем самым они увеличивают значение среднего арифметического.

Также странно, что значение медианы является отрицательным. Конечно, это связано с тем, что большинство значений в столбце являются отрицательными. Попробуем вывести только положительные значения столбца `days_employed`.

In [153]:
#вывод первых 10 строк, в которых значение столбца days_employed больше нуля
df[df['days_employed'] > 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


In [154]:
#вывод минимального значения days_employed, при условии days_employed > 0
df[df['days_employed'] > 0]['days_employed'].min()

328728.72060451825

Уже по минимальному значению в столбце `days_employed` (при условии days_employed > 0) видно, что значения являются аномальными, так как, если `days_employed` выражено в днях, то 328728/365 = 900 лет рабочего стажа. Конечно, такое невозможно, поэтому этот случай надо дополнительно исследовать.

В выборке, полученной с помощью кода `df[df['days_employed'] > 0].head(10)`, заметно, что аномальные значения обозначены только у пенсионеров. Проверим эту гипотезу.

In [155]:
#вывод количества значений income_type при условии days_employed больше нуля
df[df['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [156]:
#вывод количества пенсионеров у которых не определен рабочий стаж
df[(df['income_type'] == 'пенсионер') & (df['days_employed'].isna())]['income_type'].count()

413

In [157]:
#вывод общего количества пенсионеров
df[df['income_type'] == 'пенсионер']['income_type'].count()

3856

Видно, что с данными рабочего стажа явно что-то не так.

В них существует 2 проблемы:
- Почти все строки имеют отрицательные значения рабочего стажа.
- У всех пенсионеров, для которых определены значения рабочего стажа, значения являются положительными, но при этом сильно завышены относительно всех остальных групп пользователей.

Возможно, значения для пенсионеров представлены в часах. Чтобы устранить данную аномалию, разделим каждое значение, которое больше 0, на 24, а отрицательные значения возьмем по модулю с помощью метода abs().

In [158]:
def result_in_days(hours):
    if hours < 0:
        return abs(hours)
    return hours / 24

In [159]:
df['days_employed'] = df['days_employed'].apply(result_in_days)

In [160]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [161]:
#замена пропусков на медиану столбца days_employed
df.loc[df['days_employed'].isna(), 'days_employed'] = df['days_employed'].median()

In [162]:
#вывод количества пропусков в столбце days_employed
df['days_employed'].isna().sum()

0

Видно, что мы избавились от пропусков в столбце `days_employed`, теперь очередь столбца `total_income`. Проверим на какое значение лучше производить замену: на медиану или на среднее арифметическое.

In [163]:
#подсчет среднего арифметического
df['total_income'].mean()

167422.30220817294

In [164]:
#подсчет медианы
df['total_income'].median()

145017.93753253992

Видно, что медиана и среднее арифметическое не сильно отличаются друг от друга. Это значит, что в столбце `total_income` не содержится значений, которые сильно разнились бы от общей массы или таких значений содержится немного. Поэтому в данном случае корректнее выбрать среднее арифметическое в качестве замены пропусков.

In [165]:
#замена пропусков на среднее арифметическое
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].mean()

In [166]:
#вывод количества пропусков в столбце total_income
df['total_income'].isna().sum()

0

**Вывод**

В данном пункте были выполнены следующие действия:
1. Определены столбцы, в которых присутствовали пропуски.
2. Определена связь между этими пропусками и указана возможная причина их появления.
3. Определены возможные аномалии данных в столбце `days_employed`.
4. Пропуски в столбце `days_employed` были заменены на значение медианы этого столбца.
5. Пропуски в столбце `total_income` были заменены на значение среднего арифметического этого столбца.

**Исправление аномалий данных в столбце `dob_years`**

Ранее мы обнаружили, что в столбце `dob_years` содержатся значения 0. Для исправления этой аномалии предварительно сгруппируем данные по столбцу `family_status` и после этого заменим значения 0 на значения медианы, соответствующей группы.

In [167]:
#создание новой переменной med_family_status и запись в нее значений медианы для каждой группы
med_family_status = df.groupby('family_status')['dob_years'].median()

In [168]:
#вывод созданной переменной для проверки
med_family_status

family_status
Не женат / не замужем    36
в разводе                46
вдовец / вдова           58
гражданский брак         41
женат / замужем          43
Name: dob_years, dtype: int64

In [169]:
#цикл для замены всех значений 0 на значения медианы для каждой группы
for i in med_family_status.index:
    df.loc[(df['family_status'] == i) & (df['dob_years'] == 0), 'dob_years'] = med_family_status[i]

In [170]:
#проверка на значения 0 в столбце dob_years
df[df['dob_years'] == 0]['dob_years'].count()

0

Исправление аномалий данных в столбце `gender`

In [171]:
#вывод всех уникальных значений столбца gender
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце `gender` присутствует аномальное значение "XNA". Заменим его на самый встречаемый в исходном датафрейме пол, то есть на "F".

In [172]:
#Замена значения XNA на F
df.loc[(df['gender'] == 'XNA'), 'gender'] = 'F'

In [173]:
#проверка устранения аномалии в столбце gender
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

### Замена типа данных

В задании по устранению пропусков было заметно, что данные, содержащиеся в столбцах `days_employed` и `total_income` визуально очень тяжело анализировать, так как они записаны в вещественном виде. Попробуем преобразовать эти значения в int.

В pandas существует 2 метода для преобразования значений в число:
- to_numeric();  
- astype().

Особенность метода to_numeric() в том, что при переводе все числа будут иметь тип данных float, поэтому в этой ситуации данный метод нам не подходит. Для перевода в тип данных int необходимо использовать метод .astype('int').

In [174]:
#смена типа данных с float на int в столбце days_employed
df['days_employed'] = df['days_employed'].astype('int')

In [175]:
#вывод первых 10 строк датафрейма для просмотра измененных данных в столбце days_employed
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [177]:
df['total_income'] = df['total_income'].astype('int')

In [178]:
#вывод первых 10 строк датафрейма для просмотра измененных данных в столбце total_income
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [179]:
#вывод общей информации по датафрейму для просмотра измененных типов данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Тип данных успешно изменен.

**Вывод**

В данном пункте мы заменили тип данных в столбцах `days_employed` и `total_income` с вещественного на целочисленный с помощью метода .astype('int'). Теперь данные легче визуально анализировать. 

### Обработка дубликатов

При первичном анализе датафрейма было заметно, что в столбце `education` нет единого формата записи уровня образования. Выясним сколько различных интерпретаций уровней образования содержатся в датафрейме. Для это применим метод .value_counts() к столбцу `education`. Этот метод позволяет вывести частоту встречаемости каждого уникального значения столбца.

In [180]:
#вывод неявных дубликатов в столбце education
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что для одного и того же уровня образования существует множество вариантов написания, например, среднее, СРЕДНЕЕ, Среднее и т.д. Скорее всего такая проблема возникла из-за того, что в системе ввода данных нет установленного формата для уровня образования, поэтому каждый оператор вводит эту информацию так, как считает нужным. Конечно, такого быть не должно, поэтому необходимо перезаписать значения в столбце `education` в нижнем регистре, тем самым удалив неявные дубликаты.

In [181]:
#перевод значений столбца education в нижний регистр
df['education'] = df['education'].str.lower()

In [182]:
#проверка изменения регистра
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Сменив регистр, мы избавились от неявных дубликатов в столбце `education`.

**Вывод**

В столбце `education` были обнаружены неявные дубликаты. Они вызваны тем, что в системе не существует единой формы записи уровня образования и операторы вводят информацию в ручную, в произвольном формате. Их удалось обнаружить благодаря методу .value_counts(). Для устранения таких дубликатов необходимо привести данные к единому формату, то есть преобразовать их к нижнему регистру с помощью метода .str.lower(). 

In [183]:
#подсчет количества дубликатов
df.duplicated().sum()

72

In [184]:
#удаление дубликатов с созданием новых индексов
df = df.drop_duplicates().reset_index(drop = True)

In [185]:
#проверка количества дублей
df.duplicated().sum()

0

### Лемматизация

Первичный анализ данных показал, что в столбце `purpose` содержится большое количество однообразных целей оформления кредита, интерпретированных по-разному. Например, провести свадьбу и сыграть свадьбу - являются одинаковыми целями кредита, сформулированными разными способами. Нерегламентированная запись одних и тех же целей кредита является очень затрудняющим фактором для проведения анализа.

В данном случае необходимо провести процесс лемматизации каждой строки целей оформления кредита, то есть преобразовать каждое слово к его словарной форме. Цели кредита, записанные в словарной форме, значительно легче категорировать и анализировать, отвечая на вопрос о связи между различными целями кредита и его выплат в установленный срок.

Для проведения процесса лемматизации, необходимо подключить библиотеку pymestem3  и с помощью метода apply применить функцию lemmatize к каждой строке столбца `purpose`.

In [186]:
#подключение библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()
#создадим новый столбец purpose_lemmas и запишем в него леммы столбца purpose 
df['purpose_lemmas'] = df['purpose'].apply(m.lemmatize)

In [187]:
#вывод первых 10 строк обновленного датафрейма
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


В датайрейм был добавлен новый столбец `purpose_lemmas`, в котором содержится цель оформления кредита в лемматизированном формате.

Проверим, существует ли необходимость лемматизировать другие столбцы датафрейма.

In [188]:
#Проверка актуальности лемматизации столбца family_status
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4150
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [189]:
#Проверка актуальности проведения лемматизации столбца income_type
df['income_type'].value_counts()

сотрудник          11083
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Столбцы `family_status` и `income_type` нет необходимости лемматизировать, так как данные в них записанны в одинаковом формате.

In [190]:
#вывод первых 5 строк датафрейма с целю проверки создания столбца purpose_lemmas
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

В данном пункте проведена лемматизация столбца `purpose`. Такая процедура необходима для дальнейшего категорирования данных и ответа на вопрос о связи между целью оформления кредита и его выплатой в срок. Лемматизация осуществляется с помощью библиотеки pymystem3 и функции lemmatize.

### Категоризация данных

Для ответа на следующие вопросы:

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- Как разные цели кредита влияют на его возврат в срок?

Необходимо произвести категорирование данных.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на данный вопрос необходимо категорировать данные по уровню дохода (ниже среднего, средний, выше среднего). В данном случае крайне важно корректно выбрать граничные значения для каждой группы, так как в дальнейшем этот фактор будет непосредственно влиять на результат анализа. Для определения границ каждой группы будем отталкиваться от значений минимального, максимального и среднего арифметического столбца `total_income`.

In [191]:
#выявление минимального значения столбца total_income
df['total_income'].min()

20667

In [192]:
#выявление максимального значения столбца total_income
df['total_income'].max()

2265604

In [193]:
#выявление среднего арифметического значения столбца total_income
df['total_income'].mean()

167421.82114389597

Выполним первичный анализ квартилий столбца `total_income` с помощью функции describe().

In [194]:
df['total_income'].describe()

count    2.145300e+04
mean     1.674218e+05
std      9.779664e+04
min      2.066700e+04
25%      1.076200e+05
50%      1.560410e+05
75%      1.958180e+05
max      2.265604e+06
Name: total_income, dtype: float64

Данные в столбце `total_income` разделяются на следующие квартилии:
- от 20 667 до 107 620 (от 0% до 25%);
- от 107 620 до 156 041 (от 25% до 50%);
- от 156 041 до 195 818 (от 50% до 75%);
- от 195 818 до 2265 604 (от 75% до 100%).

Запишем диапазоны зарплат в отдельную переменную и в дальнейшем будем обращаться к ней при работе функции.

In [195]:
#применение функции pd.qcut к столбцу total_income
df_qcut = pd.qcut(df['total_income'], q=4)

In [196]:
#вывод созданной переменной
df_qcut

0        (195818.0, 2265604.0]
1         (107620.0, 156041.0]
2         (107620.0, 156041.0]
3        (195818.0, 2265604.0]
4         (156041.0, 195818.0]
                 ...          
21448    (195818.0, 2265604.0]
21449     (107620.0, 156041.0]
21450    (20666.999, 107620.0]
21451    (195818.0, 2265604.0]
21452    (20666.999, 107620.0]
Name: total_income, Length: 21453, dtype: category
Categories (4, interval[float64]): [(20666.999, 107620.0] < (107620.0, 156041.0] < (156041.0, 195818.0] < (195818.0, 2265604.0]]

In [197]:
#удаление повторяющихся значений в переменной df_qcut
df_qcut = pd.qcut(df['total_income'], q=4).unique()

In [198]:
#вывод конечных переменных для работы функции
df_qcut

[(195818.0, 2265604.0], (107620.0, 156041.0], (156041.0, 195818.0], (20666.999, 107620.0]]
Categories (4, interval[float64]): [(20666.999, 107620.0] < (107620.0, 156041.0] < (156041.0, 195818.0] < (195818.0, 2265604.0]]

Выполним категорирование, основываясь на квартилии:
- низкий (от 0% до 25%);
- средний (от 25% до 50%);
- выше среднего (от 50% до 75%);
- высокий (от 75% до 100%)

In [199]:
#функция категорирования данных по уровню дохода
def salary_category(salary):
    for i in range(len(df_qcut)):
        if (salary in df_qcut[i]) and (i == 0): #категория дохода от 195 818 до 2 265 604 
            return 'высокий'
        if (salary in df_qcut[i]) and (i == 1): #категория дохода от 107 620 до 156 041
            return 'средний'
        if (salary in df_qcut[i]) and (i == 2): #категория дохода от от 156 041 до 195 818
            return 'выше среднего'
        if (salary in df_qcut[i]) and (i == 3): #категория дохода от 20 667 до 107 620
            return 'низкий'

In [200]:
df['income_status'] = df['total_income'].apply(salary_category)

In [201]:
#вывод первых 10 строк датафрейма для проверки функции
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",выше среднего


In [202]:
df['income_status'].value_counts()

низкий           5364
высокий          5363
выше среднего    5363
средний          5363
Name: income_status, dtype: int64

В датафрейме появился новый столбец `income_status` с соответствующей категорией столбца `total_income`.

**Как разные цели кредита влияют на его возврат в срок?**

Для данного категорирования целей кредита необходимы леммы, которые мы выделяли в предыдущем пункте задания. Реализуем категорироание на основе наличия определённых лемм в каждой строчке столбца `purpose_lemmas` и разобьём данные на следующие группы:

- Образование (если в ячейке есть лемма "образование")
- Жилье (если в ячейке есть леммы "жилье" или "недвижимость")
- Свадьба (если в ячейке есть лемма "свадьба")
- Автомобиль (если в ячейке есть лемма "автомобиль")

Для этого напишем соответсвующую функцию.

In [203]:
#функция категорирования цели оформления кредита
def purpose_category(purpose_lemma):
    if 'автомобиль' in purpose_lemma:
        return 'автомобиль'
    
    if 'свадьба' in purpose_lemma:
        return 'свадьба'
    
    if 'образование' in purpose_lemma:
        return 'образование'
    
    if ('недвижимость' in purpose_lemma) or ('жилье' in purpose_lemma):
        return 'недвижимость'

In [204]:
#применение функции purpose_category к столбцу purpose_lemmas
df['purpose_category'] = df['purpose_lemmas'].apply(purpose_category)

In [205]:
#вывод первых 10 строк измененного датафрейма
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_status,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",средний,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",высокий,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",выше среднего,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",высокий,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",средний,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",низкий,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",средний,недвижимость


**Вывод**

В датафрейме появился новый столбец `purpose_category` с указанием цели оформления кредита. Теперь, основываясь на данном столбце, намного легче проводить анализ для проверки связи разных целей кредита и его своевременного возврата.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Для ответа на данный вопрос выполним категорирование на две ветки:
- Есть дети/нет детей.
- Без детей/малодетная семья(кол-во детей < 3)/многодетная семья(кол-во детей >= 3).

Но в первую очередь выясним, какие уникальные значения в столбце children существуют. Для этого применим функцию .value_counts().

In [206]:
#вывод уникальных значений в столбце children
df['children'].value_counts()

 0     14090
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В глаза сразу бросаются два значения: 20 и -1. Значения 20 детей больше похоже на ошибку системы при записи, чем на реальные данные. А значение -1 выглядит как-будто у пользователей не было желания указывать количество детей. Заменим значение 20 на 2, а -1 на 1.

In [207]:
#замена значения -1 на 1
df.loc[(df['children'] == -1), 'children'] = 1

In [208]:
#замена значения 20 на 2
df.loc[(df['children'] == 20), 'children'] = 2

In [209]:
#итоговый список уникальных зачений в столбце children
df['children'].value_counts()

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [210]:
#функция категорирования пользователей на группы есть дети/нет детей
def family_children_status(children):
    if children > 0:
        return 'есть дети'
    return 'нет детей'

In [211]:
#применение функции family_children_status к столбцу children
df['family_children_status'] = df['children'].apply(family_children_status)

In [212]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_status,purpose_category,family_children_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний,автомобиль,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",средний,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",высокий,образование,есть дети
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",выше среднего,свадьба,нет детей


In [213]:
#функция категорирования пользователей на группы без детей/малодетная семья/многодетная семья
def family_childrens_count_status(children):
    if children == 0:
        return 'без детей'
    if (0 < children) and (children <= 2):
        return 'малодетная семья'
    return 'многодетная семья'

In [214]:
#применение функции family_childrens_count_status
df['family_childrens_count_status'] = df['children'].apply(family_childrens_count_status)

In [215]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_status,purpose_category,family_children_status,family_childrens_count_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость,есть дети,малодетная семья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний,автомобиль,есть дети,малодетная семья
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",средний,недвижимость,нет детей,без детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",высокий,образование,есть дети,многодетная семья
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",выше среднего,свадьба,нет детей,без детей


В данном пункте задания было создано 2 новых столбца `family_children_status` (с категоризацией на группы "есть дети"/"нет детей") и `family_childrens_count_status` (с категоризацией на группы "без детей"/"малодетная семья"/"многодетная семья").

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос сгруппируем пользователей по количеству детей и выведем среднее арифметическое задолженности по кредиту для каждой группы.

Для более комфортного анализа создадим сводную таблицу, в которой будут представлены значения общего количества пользователей, соответствующих той или иной группе (count), суммарное количество должников по каждой группе (sum) и их среднее арифметическое (mean).

In [217]:
#создание сводной таблицы df_group_by_children_status
#сгруппируем данные по столбцу children
#применим к сгруппированным данным методы sum, count и mean
df_group_by_children_status = df.groupby('family_children_status').agg({'debt': ['sum', 'count', 'mean']})

Выведем новый датафрейм `df_group_by_children` проведем первичный анализ.

In [218]:
#вывод датафрейма df_group_by_children_status
df_group_by_children_status

debt                 
                         sum  count      mean
family_children_status                       
есть дети                678   7363  0.092082
нет детей               1063  14090  0.075444

Значения столбца `mean` представлены в виде вещественного числа. Для визуального анализа было бы комфортнее, если бы результаты среднего были представлены в процентах. Для этого создадим функцию, которая принимает на вход значение столбца `mean` и умножает его на 100. 

In [219]:
#функция перевода среднего в проценты
def mean_in_percent(mean_value):
    return mean_value * 100

Далее создадим новый столбец `mean_percent` в датафрейме df_group_by_children и с помощью метода .apply() применим функцию mean_in_percent к столбцу `mean`.

In [220]:
#применение метода mean_percent
df_group_by_children_status['mean_percent'] = df_group_by_children_status['debt']['mean'].apply(mean_in_percent)

In [221]:
df_group_by_children_status

debt                  mean_percent
                         sum  count      mean             
family_children_status                                    
есть дети                678   7363  0.092082     9.208203
нет детей               1063  14090  0.075444     7.544358

Показатели среднего арифметического датафрейма df_group_by_children_status подтверждают гипотезу о том, что наличие детей действительно влияет на выплату кредита в срок. У клиентов, у которых нет детей, при выборке в 2 раза большей, процент своевременной выплаты кредита значительно лучше относительно клиентов с детьми (**7.5%** и **9.2%** соответственно).

Это является вполне ожидаемым, так как с появлением детей у людей появляются значительные траты и выплата кредита может уйти на второй план.

Проанализируем относительно групп "без детей"/"малодетная семья"/"многодетная семья" и выявим, какая группа имеет наилучшие показатели выплаты кредита в срок.

In [222]:
#создание сводной таблицы family_childrens_count_status
#сгруппируем данные по столбцу children
#применим к сгруппированным данным методы sum, count и mean
family_childrens_count_status = df.groupby('family_childrens_count_status').agg({'debt': ['sum', 'count', 'mean']})

In [223]:
#применение метода mean_percent
family_childrens_count_status['mean_percent'] = family_childrens_count_status['debt']['mean'].apply(mean_in_percent)

In [224]:
family_childrens_count_status

debt                  mean_percent
                                sum  count      mean             
family_childrens_count_status                                    
без детей                      1063  14090  0.075444     7.544358
малодетная семья                647   6983  0.092654     9.265359
многодетная семья                31    380  0.081579     8.157895

**Вывод**

В сводной таблице df_group_by_children_status видно, что наличие детей, безусловно, влияет на своевременность выплаты кредита.

Но таблица family_childrens_count_status представляет более интересные результаты. Согласно данной таблице, малодетная семья (кол-во детей <3) имеет худший показатель кредита относительно других групп, хотя факторов, способных повлиять на задолженность по кредиту, у них меньше, чем у многодетных детей.

Это может быть связанно с тем, что выборка "малодетная семья" более многочисленная, чем "многодетная семья".

Это также может быть связано с тем, что, возможно, группа "многодетная семья" более трезво оценивает свои возможности и тщательнее анализирует все риски не выплаты кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос проделаем такие же действия, как и в предыдущем вопросе, то есть сгруппируем данные по столбцу `family_status`, применим функции sum, count и mean и выведем результат.

In [51]:
#создание сводной таблицы df_group_by_children
#сгруппируем данные по столбцу family_status
#применим к сгруппированным данным методы sum, count и mean
df_group_by_family_status = df.groupby('family_status').agg({'debt': ['sum', 'count', 'mean']})

In [52]:
df_group_by_family_status

debt                 
                       sum  count      mean
family_status                              
Не женат / не замужем  274   2813  0.097405
в разводе               85   1195  0.071130
вдовец / вдова          63    960  0.065625
гражданский брак       388   4177  0.092890
женат / замужем        931  12380  0.075202

In [53]:
#применение функции перевода среднего арифметического в проценты
df_group_by_family_status['mean_percent'] = df_group_by_family_status['debt']['mean'].apply(mean_in_percent)

In [54]:
df_group_by_family_status

debt                  mean_percent
                       sum  count      mean             
family_status                                           
Не женат / не замужем  274   2813  0.097405     9.740491
в разводе               85   1195  0.071130     7.112971
вдовец / вдова          63    960  0.065625     6.562500
гражданский брак       388   4177  0.092890     9.288963
женат / замужем        931  12380  0.075202     7.520194

**Вывод**

У групп "женат / замужем" и "в разводе" примерно одинаковые показатели (**7.5%** и **7.1%**). При том, что группа "женат / замужем" является самой многочисленной (12 380 пользователей, больше половины всей выборки).

Максимальные показатели задолженности по кредиту у групп "Не женат / не замужем" и "гражданский брак" (**9.7%** и **9.2%** соответственно). 

Самый минимальный показатель задолженности по кредиту у группы "вдовец / вдова" (**6.6%**).

В данном вопросе можно с уверенностью утверждать, что группа "женат / замужем" имеет наилучший показатель платежеспособности по кредиту. Также можно утверждать, что у группы пользователей "гражданский брак" плохой процент своевременной выплаты при достаточно большой выборке данных (4177 человек).

Насчет других групп тяжело сказать, что связь между семейным положением и выплатой кредита действительно существует, так как группы "женат / замужем" и "гражданский брак" забирают на себя почти 77% всей выборки.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выполним все те же действия, сгруппировав пользователей в сводную таблицу и применив ко всем группа функции sum, count и mean.

In [225]:
#создание сводной таблицы df_group_by_children
#отфильтруем значения с помощью логической индексации (уберем значение группы "Не определен" из общей выборки)
#сгруппируем данные по столбцу income_status
#применим к сгруппированным данным методы sum, count и mean
df_group_by_income_status = df.groupby('income_status').agg({'debt': ['sum', 'count', 'mean']})

In [226]:
#применим функцию перевода среднего в проценты
df_group_by_income_status['mean_percent'] = df_group_by_income_status['debt']['mean'].apply(mean_in_percent)

In [227]:
df_group_by_income_status

debt                 mean_percent
               sum count      mean             
income_status                                  
высокий        383  5363  0.071415     7.141525
выше среднего  463  5363  0.086332     8.633228
низкий         427  5364  0.079605     7.960477
средний        468  5363  0.087265     8.726459

**Вывод**

Результаты таблицы df_group_by_income_status получились противоречивыми. Так как при абсолютно одинаковой по количеству выборке, клиенты с низким уровнем дохода имеют чуть ли не лучший процент выплаты кредита. Выше только у клиентов с высоким уровнем дохода.

Возможно, клиенты со средним доходом и выше среднего сильно переоценивают свой финансовый достаток и берут кредит на более крупные суммы, чем клиенты низким доходом, и в процессе выплаты просто не могут потянуть обязательный платеж.

- Как разные цели кредита влияют на его возврат в срок?

Вот мы и дошли до пункта, для которого необходимо было сделать лемматизацию целей оформления кредита и категорирование причин на разные группы.

Создадим сводную таблицу с группировкой по столбцу `purpose_category`.

In [59]:
#созданеи сводной таблицы df_group_by_credit_purpose
df_group_by_credit_purpose = df.groupby('purpose_category').agg({'debt': ['sum', 'count', 'mean']})

In [60]:
#перевод значений среднего арифметического в проценты
df_group_by_credit_purpose['mean_percent'] = df_group_by_credit_purpose['debt']['mean'].apply(mean_in_percent)

In [61]:
df_group_by_credit_purpose

debt                  mean_percent
                  sum  count      mean             
purpose_category                                   
автомобиль        403   4315  0.093395     9.339513
недвижимость      782  10840  0.072140     7.214022
образование       370   4022  0.091994     9.199403
свадьба           186   2348  0.079216     7.921635

**Вывод**

Наибольший процент задолженности имеют пользователи, которые оформляют кредит для покупки автомобиля или получения образования (**9.3%** и **9.1%** соответственно). В этих группах содержатся относительно одинаковое количество пользователей, поэтому можно утверждать, что клиенты этих групп имеют одинаковую платежеспособность. 

Лучше ситуация у людей, которые берут кредит с целью проведения свадьбы (**7.9%** задолженности).

Несмотря на то, что клиентов, которые оформляют кредит на покупку недвижимости, больше всех относительно других групп, процент задолженности у них самый минимальный **7.2%**.

## Шаг 4. Общий вывод

Из этого исследования можно сделать следующие выводы:

**1. Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Появление хотя бы одного ребёнка понижает платёжеспособность клиента.

**2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Группа "женат / замужем" имеет наилучший показатель выплаты кредитов в срок. Группы "гражданский брак" и "Не женат / не замужем" имеют относительно плохой процент выплаты кредитов.

Про группы "вдовец / вдова" и "в разводе" нельзя точно утверждать о достоверности результатов, так как численность этих групп относительно небольшая.

**3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Доход клиента не является ключевым фактором платежеспособности по кредиту. Это доказывает факт того, что у клиентов со средним уровнем дохода и с доходом выше среднего процент задолженности выше, чем у клиентов с низким уровнем дохода.

**4. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Статистика показывает, что клиенты, оформляющие кредит с целью получения образования или покупки автомобиля, обладают относительно низкой платежеспособностью.

У клиентов, оформлявших кредит для проведения свадьбы, статистика платежеспособности лучше, но и численность этой выборки меньше, чем у всех остальных.

Клиенты, оформляющие кредиты с целью каких-либо манипуляций с недвижимостью имеют безусловно лучший показатель платежеспособности.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.